In [16]:
from sklearn.feature_selection import SelectKBest, mutual_info_regression
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
INCLUDE_MATRICES_AT_END = True

# Validate that all appropriate subjects are present and aligned correctly
ydf = pd.read_csv("./dependent.csv")
causality_df = pd.read_csv("./granger.csv")
display(len(causality_df['ID'][0]))


priori_metrics = pd.read_csv("./priori_metrics.csv")
subset = pd.merge(ydf[['ID']], priori_metrics, how='inner', on='ID').sort_values(by=['ID']).reset_index(drop=True)
display(subset)
subset.to_csv("subset_temp.csv", index=False)

raise SystemExit("Stop right there!")


ydf = pd.merge(ydf, causality_df, how='inner', on='ID').sort_values(by=['ID']).reset_index(drop=True)
# display(ydf)
y = ydf.end_avg_minus_start_avg.values
X = causality_df.loc[:, causality_df.columns != 'ID'].to_numpy()

# display(X)
# = np.load("connectomes.npy")



#scaling features
scaler = preprocessing.StandardScaler().fit(X)
X = scaler.transform(X)

# Split the dataset in two parts
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.15)

# Set the parameters by cross-validation
feats = len(X[0])
subs = len(y)

pipe = Pipeline([
    # the reduce_dim stage is populated by the param_grid
    ('reduce_dim', SelectKBest(mutual_info_regression)),
    ('svr', SVR(verbose=10))
])

#setting feature_selection params appropriately
if INCLUDE_MATRICES_AT_END:
    N_FEATURES_OPTIONS = [100]
else:
    N_FEATURES_OPTIONS = [5, 100, 200, "all"]

C_OPTIONS = [0.0001, 0.001, 0.1]
C_OPTIONS.reverse()
param_grid = [
    {
        'reduce_dim__k': N_FEATURES_OPTIONS,
        'svr__kernel': ['linear'],
        'svr__C': C_OPTIONS
    }
]

grid = GridSearchCV(pipe, cv=5, n_jobs=-1, param_grid=param_grid, iid=False, verbose=10)
grid.fit(X_train, y_train)

print("Best parameters set found on development set:")
print()
print(grid.best_params_)
print()
print("Grid scores on development set:")
print()
means = grid.cv_results_['mean_test_score']
stds = grid.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, grid.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
          % (mean, std * 2, params))
print()

print("Detailed classification report:")
print()
print("The model is trained on", len(y_train), "subjects.")
print("The scores are computed on", len(y_test), "subjects.")
print()
y_true, y_pred = y_test, grid.predict(X_test)
print(r2_score(y_true, y_pred))


9

,ID,bids_age,NFB3_MRIQ_01,NFB3_MRIQ_02,AGE,AGE_04,SUB_TYPE,SUB_STUDY,DIAG_01#CODE,DIAG_01#DESC,...,INT_08,INT_09,INT_10,INT_11,INT_12,INT_13,INT_14,INT_15,INT_16,INT_17
0,A00028185,26.0,1.0,0.0,28.0,28.772603,ENROLLED,Neurofeedbac,305,Alcohol Abuse,...,48.0,96.0,102.0,198.0,97.0,101.0,99.0,42.0,53.0,47.0
1,A00032875,37.0,1.0,1.0,40.0,40.852055,ENROLLED,LNG_ADULTFIT,300.29,Specific Phobia,...,45.0,95.0,79.0,174.0,96.0,83.0,88.0,39.0,13.0,21.0
2,A00033747,26.0,1.0,0.0,27.0,27.926027,ENROLLED,Neurofeedbac,V71.09,No Diagnosis or Condition on Axis I,...,53.0,94.0,84.0,178.0,95.0,87.0,89.0,37.0,19.0,23.0
3,A00034854,21.0,0.0,0.0,23.0,23.868493,ENROLLED,Neurofeedbac,296.2,"Major Depressive Disorder, Single Episode, Uns...",...,52.0,111.0,94.0,205.0,109.0,95.0,102.0,73.0,37.0,55.0
4,A00035072,36.0,0.0,0.0,38.0,38.435616,Adult- Age 22-59,Discoverysci,305,Alcohol Abuse,...,44.0,94.0,96.0,190.0,95.0,96.0,95.0,37.0,39.0,37.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,A00066827,26.0,1.0,0.0,26.0,26.786301,ONE_DAY,Neurofeedbac,305,Alcohol Abuse,...,55.0,111.0,90.0,201.0,109.0,92.0,100.0,73.0,30.0,50.0
134,A00066926,23.0,0.0,0.0,23.0,23.400000,ONE_DAY,Neurofeedbac,303.9,Alcohol Dependence,...,44.0,89.0,103.0,192.0,91.0,102.0,96.0,27.0,55.0,39.0
135,A00072203,23.0,0.0,0.0,23.0,23.194521,ONE_DAY,Neurofeedbac,300.29,Specific Phobia,...,48.0,94.0,108.0,202.0,95.0,106.0,101.0,37.0,66.0,53.0
136,A00073600,22.0,0.0,0.0,22.0,22.778082,ONE_DAY,Neurofeedbac,296.36,"Major Depressive Disorder, Recurrent, In Full ...",...,45.0,91.0,107.0,198.0,93.0,105.0,99.0,32.0,63.0,47.0


SystemExit: Stop right there!

/home/rad/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3327: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
